# Exemplo: Validando o U-Classifier em Dados Reais

Este notebook demonstra como usar as funções do pacote `uclassifier` para testar o classificador em um dataset real, replicando a metodologia de validação cruzada descrita no artigo de Ahmad & Pavlenko (2018) para os dados de Shipp et al. (2002).

**Objetivo:**
1. Carregar um dataset de um arquivo CSV.
2. Executar uma validação cruzada de 3-folds, repetida 100 vezes para estabilidade.
3. Calcular e analisar as taxas de erro APER, `e(1|0)` e `e(0|1)`.

In [1]:
# Importa as funções necessárias do pacote e de outras bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

# Importa a função principal do nosso pacote
from u_classifier_hd.core import u_classifier_multisample

## Passo 1: Carregar e Preparar os Dados
Vamos carregar o dataset de exemplo `shipp_data_sample.csv`, que simula a estrutura do dataset original. Substitua o `filepath` pelo caminho do seu próprio arquivo de dados.

In [2]:
data_df = pd.read_csv("../data/dados_shipp.csv")

# Separa as características (X) da variável alvo (y)
X = data_df.drop(columns=['y']).values
y = data_df['y'].values

# Converte as classes de texto para números: 0 (DLBCL) e 1 (FL)
classes, y_numeric = np.unique(y, return_inverse=True)
class_map = {label: i for i, label in enumerate(classes)}

In [3]:
print("Dataset Info:")
print(f"p: {data_df.shape[1]} gene expressions")
print(f"N: {data_df.shape[0]} patients")
print(f"    one with DLBCL: {data_df.y.value_counts()['DLBCL']}")
print(f"    the other with follicular lymphoma (FL): {data_df.y.value_counts()['FL']}")

Dataset Info:
p: 7130 gene expressions
N: 77 patients
    one with DLBCL: 58
    the other with follicular lymphoma (FL): 19


## Passo 2: Executar a Validação Cruzada com Múltiplas Réplicas

In [4]:
#For a 3-fold CV, we randomly divide the data into three groups of sizes 26,26, 25 with n1(L) = 52, n1(T ) = 25 and nk(L) = 51, nk(T ) = 26 for k = 2; 3.
n_splits = 3
all_fold_results = []
n_replicas = 100

In [5]:
for replica in range(n_replicas):
    # Usar um random_state diferente para cada réplica garante novas divisões dos dados
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=replica)
    
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y_numeric), 1):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y_numeric[train_idx], y_numeric[test_idx]

        n1_L = np.sum(y_train == 0)
        n2_L = np.sum(y_train == 1)
        n1_T = np.sum(y_test == 0)
        n2_T = np.sum(y_test == 1)
        
        train_samples_by_class = [X_train[y_train == i] for i in range(len(classes))]
        
        m12 = 0 # Erro: Verdadeiro é classe 0 (DLBCL), previsto como 1 (FL)
        m21 = 0 # Erro: Verdadeiro é classe 1 (FL), previsto como 0 (DLBCL)
        
        for i in range(len(X_test)):
            true_class_idx = y_test[i]
            predicted_class_idx = u_classifier_multisample(X_test[i], train_samples_by_class, return_scores=False)
            
            if predicted_class_idx != true_class_idx:
                if true_class_idx == 0 and predicted_class_idx == 1:
                    m12 += 1
                elif true_class_idx == 1 and predicted_class_idx == 0:
                    m21 += 1

        aper_fold = (m12 + m21) / len(y_test) if len(y_test) > 0 else 0
        e10_fold = m12 / n1_T if n1_T > 0 else 0
        e01_fold = m21 / n2_T if n2_T > 0 else 0

        all_fold_results.append({
            'replica': replica + 1,
            'fold': fold,
            'n1_L': n1_L,
            'n2_L': n2_L,
            'n1_T': n1_T,
            'n2_T': n2_T,
            'm12': m12,
            'm21': m21,
            'aper_fold': aper_fold, 
            'e10_fold': e10_fold, 
            'e01_fold': e01_fold
        })

## Passo 3: Analisar os Resultados

In [6]:
results_df = pd.DataFrame(all_fold_results)

# Calcula as médias a partir do DataFrame detalhado
avg_aper = results_df['aper_fold'].mean()
avg_e10 = results_df['e10_fold'].mean()
avg_e01 = results_df['e01_fold'].mean()

# Calcula e imprime as médias
total_errors_per_replica = results_df.groupby('replica')[['m12', 'm21']].sum().sum(axis=1)
avg_total_errors = total_errors_per_replica.mean()

print("\n--- Resultados Médios de {} Réplicas ---".format(n_replicas))
print(f"Taxa de erro e(1|0) média (DLBCL -> FL): {avg_e10:.4f}")
print(f"Taxa de erro e(0|1) média (FL -> DLBCL): {avg_e01:.4f}")
print(f"Taxa de erro de classificação média (APER): {avg_aper:.4f} ({avg_total_errors:.0f}/{len(y)})")

article_rate = 15 / 77
print(f"\nTaxa de erro no artigo (Shipp et al.): {article_rate:.4f} (15/77)")


--- Resultados Médios de 100 Réplicas ---
Taxa de erro e(1|0) média (DLBCL -> FL): 0.2397
Taxa de erro e(0|1) média (FL -> DLBCL): 0.1679
Taxa de erro de classificação média (APER): 0.2221 (17/77)

Taxa de erro no artigo (Shipp et al.): 0.1948 (15/77)


In [7]:
print("--- Tabela de Resultados Detalhados (Primeiras 10 Linhas) ---")
print(results_df.head(10))

--- Tabela de Resultados Detalhados (Primeiras 10 Linhas) ---
   replica  fold  n1_L  n2_L  n1_T  n2_T  m12  m21  aper_fold  e10_fold  \
0        1     1    38    13    20     6    2    0   0.076923  0.100000   
1        1     2    39    12    19     7    8    4   0.461538  0.421053   
2        1     3    39    13    19     6    3    0   0.120000  0.157895   
3        2     1    38    13    20     6    4    0   0.153846  0.200000   
4        2     2    39    12    19     7    2    3   0.192308  0.105263   
5        2     3    39    13    19     6    8    1   0.360000  0.421053   
6        3     1    38    13    20     6    5    1   0.230769  0.250000   
7        3     2    39    12    19     7    5    0   0.192308  0.263158   
8        3     3    39    13    19     6    3    3   0.240000  0.157895   
9        4     1    38    13    20     6    5    0   0.192308  0.250000   

   e01_fold  
0  0.000000  
1  0.571429  
2  0.000000  
3  0.000000  
4  0.428571  
5  0.166667  
6  0.166667  
